In [2]:
# Import librerias
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

## Se inicia la exploracion de los datasets de Evaluación Nacional Aprender


* Licencia: CC-BY-2.5-AR
* Frecuencia de actualización: Anualmente
* Responsable: Secretaría de Educación 
* Fecha de publicación: 2016
* Fecha de actualización: 2024
* Fuente primaria: Datos abiertos de la Secretaría de Educación 


In [12]:
#markdown de que tratan las columnas
#hacer un grafico de una pregunta interesante (grupo de columnas) 
#primeras 15 patrick
#siguient bruno 
#34 a final juan
#https://docs.google.com/document/d/1xeXGwWcrq0DfJiaZdMEY90f4sume7BjmDSZ1k1nbNno/edit?usp=sharing
#https://data.educacion.gob.ar/aprender-respuestas-estudiante

In [118]:
ruta_archivo_leng = "../data/raw/2024 Base APRENDER - Censal - Secundaria 5-6 año - Agregada - Desempeños de Lengua(in).csv"
ruta_archivo_mate = "../data/raw/2024 Base APRENDER - Censal - Secundaria 5-6 año - Agregada - Desempeños de Matematica(in).csv"

ruta_archivo_leng_pre = "../data/interim/2024_Ap_56Lengcoma.csv"
ruta_archivo_mate_pre = "../data/processed/2024_Ap_56Matecoma.csv"

ruta_archivo_leng_pre2 = "../data/interim/2024_Ap_56Lengceros.csv"
ruta_archivo_mate_pre2 = "../data/interim/2024_Ap_56Mateceros.csv"

ruta_archivo_leng_pre3 = "../data/interim/2024_Ap_56Lenglisto.csv"
ruta_archivo_mate_pre3 = "../data/interim/2024_Ap_56Matelisto.csv"



* Debido a que el dataset tiene una codificacion latin-1 y
  utiliza el coma como punto decimal, esta funcion reemplaza por punto las comas

In [119]:
def reemplazar_comas_por_puntos(input_csv, output_csv):
    """
    Reemplaza todas las comas por puntos en un archivo CSV con separador ';'
    y guarda el resultado en un nuevo archivo.
    
    :param input_csv: Ruta del archivo CSV original
    :param output_csv: Ruta del archivo CSV de salida
    """
    with open(input_csv, 'r', encoding='latin-1') as f_in:
        contenido = f_in.read()
    
    # Reemplaza todas las comas por puntos
    contenido = contenido.replace(',', '.')
    
    with open(output_csv, 'w', encoding='utf-8') as f_out:
        f_out.write(contenido)

# Ejemplo de uso
reemplazar_comas_por_puntos(ruta_archivo_leng, ruta_archivo_leng_pre)
reemplazar_comas_por_puntos(ruta_archivo_mate, ruta_archivo_mate_pre)

In [ ]:
* Se abre el nuevo archivo generado y se comprueba su estado

In [120]:
df1 = pd.read_csv(ruta_archivo_leng_pre, sep=';', decimal=".", low_memory=False)
df2 = pd.read_csv(ruta_archivo_mate_pre, sep=';', decimal=".", low_memory=False)

In [121]:
df1.head(2)

,jurisdiccion,departamento,sector,ambito,ap01_Blanco,ap01_No_disponible,ap01_Multimarca,ap01_Enero,ap01_Febrero,ap01_Marzo,...,Nivel_Ed_Persona_Resp_Blanco,Nivel_Ed_Persona_Resp_No_disponible,Nivel_Ed_Persona_Resp_Multimarca,Nivel_Ed_Persona_Resp_No_fue_a_la_escuela,Nivel_Ed_Persona_Resp_Primaria_incompleto,Nivel_Ed_Persona_Resp_Primaria_completo,Nivel_Ed_Persona_Resp_Secundaria_incompleto,Nivel_Ed_Persona_Resp_Secundaria_completo,Nivel_Ed_Persona_Resp_Terciario_universitario_posgrado_incomple,Nivel_Ed_Persona_Resp_Terciario_universitario_posgrado_completo
0,Ciudad Autónoma de Buenos Aires,Comuna 1,Estatal,Urbano,,,,162.0171204,168.4406553,179.9559899,...,,,,11.9878283,18.4604794,18.5340397,49.8504244,85.80181,29.0259314,156.0451127
1,Ciudad Autónoma de Buenos Aires,Comuna 1,Privado,Urbano,,,,152.8266478,114.963918,112.7640288,...,,,,5.9561216,2.3819654,8.4506522,15.9379779,39.5181871,28.081318,110.2102714


* Como se observa que hay columnas vacias, se prodece a procesarlas para ponerlas 
 con el valor 0 de tipo float, dejando sin cambios las primeras 4 columnas que son categoricas

In [100]:
df1.columns

Index(['jurisdiccion', 'departamento', 'sector', 'ambito', 'ap01_Blanco',
       'ap01_No_disponible', 'ap01_Multimarca', 'ap01_Enero', 'ap01_Febrero',
       'ap01_Marzo',
       ...
       'Nivel_Ed_Persona_Resp_Blanco', 'Nivel_Ed_Persona_Resp_No_disponible',
       'Nivel_Ed_Persona_Resp_Multimarca',
       'Nivel_Ed_Persona_Resp_No_fue_a_la_escuela',
       'Nivel_Ed_Persona_Resp_Primaria_incompleto',
       'Nivel_Ed_Persona_Resp_Primaria_completo',
       'Nivel_Ed_Persona_Resp_Secundaria_incompleto',
       'Nivel_Ed_Persona_Resp_Secundaria_completo',
       'Nivel_Ed_Persona_Resp_Terciario_universitario_posgrado_incomple',
       'Nivel_Ed_Persona_Resp_Terciario_universitario_posgrado_completo'],
      dtype='object', length=1035)

In [122]:
def rellenar_y_convertir_columnas_desde_4(df: pd.DataFrame) -> pd.DataFrame:
    """
    Rellena los valores vacíos o con solo espacios con 0 y convierte a float
    todas las columnas desde la 4ta hasta la última del DataFrame.
    Evita el warning de downcasting de pandas.
    """
    columnas = df.columns[4:]
    
    for col in columnas:
        # Convertimos a string primero para que replace no genere warning
   #     df[col] = df[col].astype(str).replace(r'^\s*$', '0', regex=True)
        # Convertimos a float
  #      df[col] = pd.to_numeric(df[col], errors='coerce').fillna(0).astype(float)
        df[col] = df[col].mask(df[col].astype(str).str.strip() == '', 0)
        df[col] = pd.to_numeric(df[col], errors='coerce').fillna(0).astype(float)
    
    return df

# Ejemplo de uso
#df = pd.read_csv("archivo.csv")
df1 = rellenar_y_convertir_columnas_desde_4(df1)
df2 = rellenar_y_convertir_columnas_desde_4(df2)

In [123]:
df1.head()

,jurisdiccion,departamento,sector,ambito,ap01_Blanco,ap01_No_disponible,ap01_Multimarca,ap01_Enero,ap01_Febrero,ap01_Marzo,...,Nivel_Ed_Persona_Resp_Blanco,Nivel_Ed_Persona_Resp_No_disponible,Nivel_Ed_Persona_Resp_Multimarca,Nivel_Ed_Persona_Resp_No_fue_a_la_escuela,Nivel_Ed_Persona_Resp_Primaria_incompleto,Nivel_Ed_Persona_Resp_Primaria_completo,Nivel_Ed_Persona_Resp_Secundaria_incompleto,Nivel_Ed_Persona_Resp_Secundaria_completo,Nivel_Ed_Persona_Resp_Terciario_universitario_posgrado_incomple,Nivel_Ed_Persona_Resp_Terciario_universitario_posgrado_completo
0,Ciudad Autónoma de Buenos Aires,Comuna 1,Estatal,Urbano,0.0,0.0,0.0,162.017120,168.440655,179.955990,...,0.0,0.0,0.0,11.987828,18.460479,18.534040,49.850424,85.801810,29.025931,156.045113
1,Ciudad Autónoma de Buenos Aires,Comuna 1,Privado,Urbano,0.0,0.0,0.0,152.826648,114.963918,112.764029,...,0.0,0.0,0.0,5.956122,2.381965,8.450652,15.937978,39.518187,28.081318,110.210271
2,Ciudad Autónoma de Buenos Aires,Comuna 10,Estatal,Urbano,0.0,0.0,0.0,76.834254,102.428661,91.227815,...,0.0,0.0,0.0,4.399404,6.581900,4.550060,32.354435,58.277813,18.116752,47.907880
3,Ciudad Autónoma de Buenos Aires,Comuna 10,Privado,Urbano,0.0,0.0,0.0,114.127398,84.716258,96.843710,...,0.0,0.0,0.0,5.844474,1.305030,2.507037,22.487811,42.116628,30.763302,105.837412
4,Ciudad Autónoma de Buenos Aires,Comuna 11,Estatal,Urbano,0.0,0.0,0.0,80.486393,57.005538,68.978603,...,0.0,0.0,0.0,3.070107,9.086502,1.482637,37.686962,53.557962,12.758785,30.186418


In [124]:
df2.head()

,jurisdiccion,departamento,sector,ambito,ap01_Blanco,ap01_No_disponible,ap01_Multimarca,ap01_Enero,ap01_Febrero,ap01_Marzo,...,Nivel_Ed_Persona_Resp_Blanco,Nivel_Ed_Persona_Resp_No_disponible,Nivel_Ed_Persona_Resp_Multimarca,Nivel_Ed_Persona_Resp_No_fue_a_la_escuela,Nivel_Ed_Persona_Resp_Primaria_incompleto,Nivel_Ed_Persona_Resp_Primaria_completo,Nivel_Ed_Persona_Resp_Secundaria_incompleto,Nivel_Ed_Persona_Resp_Secundaria_completo,Nivel_Ed_Persona_Resp_Terciario_universitario_posgrado_incomple,Nivel_Ed_Persona_Resp_Terciario_universitario_posgrado_completo
0,Ciudad Autónoma de Buenos Aires,Comuna 1,Estatal,Urbano,0.0,0.0,0.0,164.371787,176.216148,178.096707,...,0.0,0.0,0.0,11.969616,18.600477,21.281081,39.496587,83.687215,24.519076,158.402045
1,Ciudad Autónoma de Buenos Aires,Comuna 1,Privado,Urbano,0.0,0.0,0.0,153.026797,114.800153,113.129972,...,0.0,0.0,0.0,5.952883,0.000000,8.288188,15.941491,39.708244,27.785983,110.507941
2,Ciudad Autónoma de Buenos Aires,Comuna 10,Estatal,Urbano,0.0,0.0,0.0,76.240297,105.053777,88.027701,...,0.0,0.0,0.0,3.050054,6.750184,5.037498,33.138638,58.556008,18.777238,48.785247
3,Ciudad Autónoma de Buenos Aires,Comuna 10,Privado,Urbano,0.0,0.0,0.0,113.496421,85.540429,97.949533,...,0.0,0.0,0.0,5.713592,1.304320,2.550951,21.050045,44.947701,31.876019,104.403566
4,Ciudad Autónoma de Buenos Aires,Comuna 11,Estatal,Urbano,0.0,0.0,0.0,81.107824,57.527755,65.355275,...,0.0,0.0,0.0,2.787548,4.512754,1.410500,39.858709,49.585557,12.983318,25.361202


## Con ambos df procesados

* se procede a guardarlos

In [125]:
def guardar_df(df: pd.DataFrame, ruta: str):
    """
    Guarda un DataFrame en un archivo CSV en la ruta indicada.

    Parámetros:
    df (pd.DataFrame): DataFrame a guardar.
    ruta (str): Ruta completa del archivo donde se guardará el CSV.
    """
    try:
        df.to_csv(ruta, index=False, encoding='utf-8')
        print(f"Archivo guardado exitosamente en: {ruta}")
    except Exception as e:
        print(f"Error al guardar el archivo: {e}")

guardar_df(df1, ruta_archivo_leng_pre2)
guardar_df(df2, ruta_archivo_mate_pre2)


Archivo guardado exitosamente en: ../data/interim/2024_Ap_56Lengceros.csv
Archivo guardado exitosamente en: ../data/interim/2024_Ap_56Mateceros.csv


In [127]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1228 entries, 0 to 1227
Columns: 1035 entries, jurisdiccion to Nivel_Ed_Persona_Resp_Terciario_universitario_posgrado_completo
dtypes: float64(1031), object(4)
memory usage: 9.7+ MB


# Dataset



* las columnas 12345 tratan de edad: 

In [126]:
from pandas.api.types import is_numeric_dtype

def analizar_dataframe(df):
    for col in df.columns:
        print(f"\nColumna: {col}")
        tipo = df[col].dtype

        if is_numeric_dtype(tipo):
            print("Tipo: float")
            print(f"  Media: {df[col].mean()}")
            print(f"  Moda: {df[col].mode().iloc[0] if not df[col].mode().empty else 'Sin moda'}")
            print(f"  Mínimo: {df[col].min()}")
            print(f"  Máximo: {df[col].max()}")

        elif pd.api.types.is_object_dtype(tipo):
            print("Tipo: objeto")
            num_categorias = df[col].nunique()
            if num_categorias > 20:
                print(f"  Más de 20 categorías ({num_categorias}), no se listan.")
            else:
                print(f"  Categorías más frecuentes:")
                print(f"  {df[col].value_counts().head(5)}")
        else:
            print(f"Tipo no explorado: {tipo}")
        print("\n")


In [17]:
#analizar_dataframe(data)

In [57]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1228 entries, 0 to 1227
Columns: 1035 entries, jurisdiccion to Nivel_Ed_Persona_Resp_Terciario_universitario_posgrado_completo
dtypes: float64(540), int64(491), object(4)
memory usage: 9.7+ MB


In [59]:
df1.dtypes

jurisdiccion                                                        object
departamento                                                        object
sector                                                              object
ambito                                                              object
ap01_Blanco                                                          int64
                                                                    ...   
Nivel_Ed_Persona_Resp_Primaria_completo                            float64
Nivel_Ed_Persona_Resp_Secundaria_incompleto                        float64
Nivel_Ed_Persona_Resp_Secundaria_completo                          float64
Nivel_Ed_Persona_Resp_Terciario_universitario_posgrado_incomple    float64
Nivel_Ed_Persona_Resp_Terciario_universitario_posgrado_completo    float64
Length: 1035, dtype: object

## Se procede a comparar ambos dataset el de lengua y matematica

In [146]:
#import pandas as pd

# Cargar ambos CSV
#df1 = pd.read_csv(ruta_archivo_leng, sep=';', decimal=",", encoding="latin-1")
#df2 = pd.read_csv(ruta_archivo_mate, sep=';', decimal=",",  encoding="latin-1")



# Comparar dimensiones (deberían ser iguales)
print("Shape archivo1:", df1.shape)
print("Shape archivo2:", df2.shape)




Shape archivo1: (1228, 1031)
Shape archivo2: (1228, 1031)


In [129]:
print(df1.columns)

Index(['jurisdiccion', 'departamento', 'sector', 'ambito', 'ap01_Blanco',
       'ap01_No_disponible', 'ap01_Multimarca', 'ap01_Enero', 'ap01_Febrero',
       'ap01_Marzo',
       ...
       'Nivel_Ed_Persona_Resp_Blanco', 'Nivel_Ed_Persona_Resp_No_disponible',
       'Nivel_Ed_Persona_Resp_Multimarca',
       'Nivel_Ed_Persona_Resp_No_fue_a_la_escuela',
       'Nivel_Ed_Persona_Resp_Primaria_incompleto',
       'Nivel_Ed_Persona_Resp_Primaria_completo',
       'Nivel_Ed_Persona_Resp_Secundaria_incompleto',
       'Nivel_Ed_Persona_Resp_Secundaria_completo',
       'Nivel_Ed_Persona_Resp_Terciario_universitario_posgrado_incomple',
       'Nivel_Ed_Persona_Resp_Terciario_universitario_posgrado_completo'],
      dtype='object', length=1035)


In [132]:
# Listar las columnas de cada uno
cols1 = set(df1.columns)
cols2 = set(df2.columns)

#print("Columnas en archivo1:", cols1)
#print("Columnas en archivo2:", cols2)

# Columnas que comparten
comunes = cols1.intersection(cols2)
#print("\n✅ Columnas comunes:", comunes)

# Columnas que están en uno pero no en el otro
solo_en_1 = cols1 - cols2
solo_en_2 = cols2 - cols1

print("\n📌 Solo en archivo1:", solo_en_1)
print("📌 Solo en archivo2:", solo_en_2)


📌 Solo en archivo1: {'ldesemp_Satisfactorio', 'ldesemp_Por_debajo_del_nivel_básico', 'ldesemp_Básico', 'ldesemp_Avanzado'}
📌 Solo en archivo2: {'mdesemp_Avanzado', 'mdesemp_Básico', 'mdesemp_Por_debajo_del_nivel_básico', 'mdesemp_Satisfactorio'}


# Diferencias encontradas

hay columnas que estan en uno de ambos datasets, por lo que le vamos a hacer un join


Esto quiere decir que esas columnas las vamos a unir con un join,
pero vamos a ver la diferencia entre las otras

Pero antes, vamos a ver si difieren en ambos datos



In [148]:


# Columnas comunes (convertir a lista ordenada)
comunes = sorted(set(df1.columns).intersection(set(df2.columns)))

#print("Columnas comunes:", comunes)

# Filtrar ambos dataframes solo con las columnas comunes
df1 = df1[comunes]
df2 = df2[comunes]

#df1 = df1.reset_index(drop=True)
#df2 = df2.reset_index(drop=True)
# Ver en qué filas/columnas difieren
diff = (df1 != df2)

# Contar cuántas celdas son distintas
total_diferencias = diff.sum().sum()
#print("Total de diferencias:", total_diferencias)

# Mostrar las filas con diferencias
diferencias = df1[diff.any(axis=1)]
#print("Filas con diferencias:")
#print(diferencias)

# Si querés ver lado a lado:
comparacion = df1.compare(df2)
print("Comparacion")
print(comparacion.head(3))


Comparacion
  NSE_Quintiles_Q1             NSE_Quintiles_Q2             NSE_Quintiles_Q3  \
              self       other             self       other             self   
0       305.454077  311.912866       395.816310  397.506189       448.543402   
1        90.850428   89.540904       141.249120  139.963330       248.742979   
2       155.514010  149.735497       163.553295  158.893421       241.305655   

              NSE_Quintiles_Q4             NSE_Quintiles_Q5              ...  \
        other             self       other             self       other  ...   
0  434.697672       406.694880  412.287567       447.722680  444.684165  ...   
1  247.529080       343.896081  350.372409       515.733006  512.337019  ...   
2  249.487679       262.597119  267.873300       214.482164  215.257795  ...   

  sobreedad_1_año_de_sobreedad_18_años_al_30_de_junio              \
                                                 self       other   
0                                         354.27

# Vista en profundidad

debido a que ambos dataset tinen datos censurados, vamos a ver que tanto difieren

la solucion sera hacer un promedio de las columnas que son iguales


In [149]:
# Supongamos que tus DataFrames se llaman df1 y df2

# Columnas comunes
columnas_comunes = df1.columns.intersection(df2.columns)

# Excluir columnas de tipo object (strings)
columnas_numericas = [col for col in columnas_comunes if pd.api.types.is_numeric_dtype(df1[col])]

# Seleccionamos solo columnas numéricas y hacemos copia
df1_comun = df1[columnas_numericas].copy()
df2_comun = df2[columnas_numericas].copy()

# Convertir números con coma decimal a float
for col in columnas_numericas:
    df1_comun[col] = df1_comun[col].astype(str).str.replace(',', '.').astype(float)
    df2_comun[col] = df2_comun[col].astype(str).str.replace(',', '.').astype(float)

# Restar DataFrames
diferencias = df1_comun - df2_comun

# Estadísticas
pd.set_option('display.max_rows', None)
print(diferencias.describe())
print("Máxima diferencia por columna:\n", diferencias.max())
#print("Máxima diferencia absoluta total:", diferencias.abs().max().max())

       NSE_Quintiles_Q1  NSE_Quintiles_Q2  NSE_Quintiles_Q3  NSE_Quintiles_Q4  \
count       1228.000000       1228.000000       1228.000000       1228.000000   
mean           0.157314         -0.053904          0.105808         -0.066229   
std            3.618871          2.867323          2.993732          2.408772   
min          -29.805805        -37.746266        -26.559587        -17.428621   
25%           -0.486343         -0.610869         -0.464470         -0.447804   
50%            0.000000          0.000000          0.000000          0.000000   
75%            0.527975          0.569087          0.700515          0.365617   
max           59.526214         23.082405         29.589872         11.669179   

       NSE_Quintiles_Q5  Nivel_Ed_Madre_Blanco  Nivel_Ed_Madre_Multimarca  \
count       1228.000000                 1228.0                     1228.0   
mean          -0.142990                    0.0                        0.0   
std            3.036386                